<a href="https://colab.research.google.com/github/bencarkaxhia/generative-ai-for-beginners/blob/main/FineTuneRomanEmpire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets torch scikit-learn

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments

# Load dataset (replace 'roman_empire_qa.csv' with your actual file)
dataset = load_dataset("csv", data_files="roman_empire_qa.csv")

# Split dataset into train and test sets
split_datasets = dataset["train"].train_test_split(test_size=0.2)

# Load pretrained tokenizer and model (replace with your preferred model)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Preprocessing function
def preprocess_function(examples):
    # Convert fields to strings explicitly
    questions = [str(q) for q in examples["Question"]]
    answers = [str(a) for a in examples["Answer"]]

    tokenized = tokenizer(
        questions,
        answers,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt",
    )
    tokenized["labels"] = tokenized["input_ids"]  # Assign input_ids as labels
    return tokenized

# Apply preprocessing to datasets
tokenized_datasets = {
    "train": split_datasets["train"].map(preprocess_function, batched=True),
    "test": split_datasets["test"].map(preprocess_function, batched=True),
}

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    remove_unused_columns=False,  # Ensure no columns are automatically dropped
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the trained model
model.save_pretrained("./roman_empire_qa_model")
tokenizer.save_pretrained("./roman_empire_qa_model")

# (Optional) Upload Model to Huggingface Hub
from huggingface_hub import login

# Login to Huggingface
login()  # Provide your Huggingface credentials

# Push model to Huggingface Hub
model.push_to_hub("roman_empire_qa_model")
tokenizer.push_to_hub("roman_empire_qa_model")

print("Fine-tuning complete! Model and tokenizer saved.")


FileNotFoundError: Unable to find '/content/roman_empire_qa.csv'

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments

# Load dataset (replace 'roman_empire_qa.csv' with your actual file)
dataset = load_dataset("csv", data_files="/content/roman_empire_qa.csv")

# Split dataset into train and test sets
split_datasets = dataset["train"].train_test_split(test_size=0.2)

# Load pretrained tokenizer and model (replace with your preferred model)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Preprocessing function
def preprocess_function(examples):
    # Convert fields to strings explicitly
    questions = [str(q) for q in examples["Question"]]
    answers = [str(a) for a in examples["Answer"]]

    tokenized = tokenizer(
        questions,
        answers,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt",
    )
    tokenized["labels"] = tokenized["input_ids"]  # Assign input_ids as labels
    return tokenized

# Apply preprocessing to datasets
tokenized_datasets = {
    "train": split_datasets["train"].map(preprocess_function, batched=True),
    "test": split_datasets["test"].map(preprocess_function, batched=True),
}

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    remove_unused_columns=False,  # Ensure no columns are automatically dropped
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the trained model
model.save_pretrained("./roman_empire_qa_model")
tokenizer.save_pretrained("./roman_empire_qa_model")

# (Optional) Upload Model to Huggingface Hub
from huggingface_hub import login

# Login to Huggingface
login()  # Provide your Huggingface credentials

# Push model to Huggingface Hub
model.push_to_hub("roman_empire_qa_model")
tokenizer.push_to_hub("roman_empire_qa_model")

print("Fine-tuning complete! Model and tokenizer saved.")


In [ ]:
# Install the neccessary libraries
!pip install transformers datasets torch pandas

# Loading and preprocessing the dataset
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the dataset
df = pd.read_csv('/content/roman_empire_qa.csv')

# Drop missing values
df = df.dropna()

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)

# Split into train and validation sets
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
val_dataset = dataset['test']

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Preprocessing function
def preprocess_function(examples):
    inputs = [f"question: {q} answer:" for q in examples['Question']]
    targets = [str(a) for a in examples['Answer']]  # Ensure targets are strings
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply preprocessing
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

# Load the model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluating and saving the model for future use
results = trainer.evaluate()
print(results)

# Save the model
model.save_pretrained('./fine-tuned-t5')
tokenizer.save_pretrained('./fine-tuned-t5')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
